In [11]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [12]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.listdir()

['.DS_Store', 'project1_gestos.ipynb', '.ipynb_checkpoints']

In [44]:
classifier = Sequential()
# Paso 1 - Convolución
classifier.add(Conv2D(filters = 64,kernel_size = (3, 3), 
                      input_shape = (64, 64, 3), activation = "relu"))
# Paso 2 - Max Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))
# Una segunda capa de convolución y max pooling
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))
# Paso 3 - Flattening
classifier.add(Flatten())
# Paso 4 - Full Connection
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 64, activation = "relu"))
classifier.add(Dense(units = 4, activation = "softmax"))
# Compilar la CNN
classifier.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
classifier.summary()

# Parte 2 - Ajustar la CNN a las imágenes para entrenar 



Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 6272)            

In [45]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('./../selfie-project/dataset_gestos/train/',
                                                    target_size=(64, 64), 
                                                    batch_size=32,
                                                    class_mode="sparse", classes=None,)

testing_dataset = test_datagen.flow_from_directory('./../selfie-project/dataset_gestos/test/',
                                                target_size=(64, 64), 
                                                batch_size=32,
                                                class_mode="sparse", classes=None,)

# Explicar esta relacion.

classifier.fit_generator(training_dataset,
                        steps_per_epoch=2864,
                        epochs=3,
                        validation_data=testing_dataset,
                        validation_steps= 1085)

Found 2864 images belonging to 4 classes.
Found 1085 images belonging to 4 classes.
Epoch 1/3
2864/2864 [==============================] - 403s 141ms/step - loss: 0.1044 - accuracy: 0.9590 - val_loss: 4.2799 - val_accuracy: 0.7078
Epoch 2/3
2864/2864 [==============================] - 405s 141ms/step - loss: 0.0152 - accuracy: 0.9954 - val_loss: 3.3629 - val_accuracy: 0.7200
Epoch 3/3
2864/2864 [==============================] - 408s 143ms/step - loss: 0.0082 - accuracy: 0.9977 - val_loss: 5.4479 - val_accuracy: 0.7067


In [49]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('./../selfie-project/dataset_gestos/test/mano_cerrado/aabdtvka.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image) 
print(result)

[[9.7395720e-12 2.5638981e-14 1.0000000e+00 4.4471979e-17]]


In [50]:
classifier.save("gestos.h5")

In [39]:
training_dataset.class_indices

{'desconocido': 0, 'mano_abierta': 1, 'mano_cerrado': 2, 'mano_dedo': 3}

In [52]:
np.argmax(result[0])

2